<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/grandes_modelos_de_lenguaje/introducci%C3%B3n_a_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción

En la era digital actual, el procesamiento del lenguaje natural ha experimentado avances significativos gracias a los modelos de aprendizaje automático, en particular, los modelos largos del lenguaje. Estos modelos, basados en técnicas de inteligencia artificial, han revolucionado la forma en que las computadoras comprenden y generan texto de manera similar a los humanos.

Un modelo largo del lenguaje es capaz de aprender patrones complejos y estructuras gramaticales en grandes cantidades de texto, adquiriendo conocimiento sobre el lenguaje en general. Estos modelos se entrenan en enormes conjuntos de datos, como libros, artículos de noticias, páginas web y corpus lingüísticos, para capturar las regularidades y matices del lenguaje humano.

### Funcionamiento

La arquitectura principal detrás de estos modelos es el Transformer, que se destaca por su capacidad para procesar secuencias de texto de manera eficiente. El Transformer utiliza capas de atención y cálculos paralelos para capturar las relaciones entre las palabras y generar representaciones contextualizadas de cada palabra en función de su contexto. Estas representaciones son fundamentales para la capacidad del modelo de comprender el significado y la estructura del texto.

Una vez que un modelo largo del lenguaje ha sido entrenado, puede ser utilizado para diversas tareas del procesamiento del lenguaje natural, como la traducción automática, la generación de texto, el resumen de documentos, el análisis de sentimientos y la respuesta a preguntas. Estos modelos son altamente flexibles y pueden adaptarse a diferentes dominios o tareas específicas.

### Limitaciones

Si bien los modelos largos del lenguaje han demostrado ser asombrosamente efectivos para muchas aplicaciones, también presentan desafíos. El entrenamiento y la utilización de estos modelos requieren una gran cantidad de recursos computacionales y de almacenamiento, así como una gran cantidad de datos de entrenamiento etiquetados. Además, la ética y la privacidad son consideraciones importantes en el uso de estos modelos, ya que pueden generar contenido potencialmente engañoso o discriminatorio si no se manejan adecuadamente. Y dificultades para comprender el razonamiento y la causalidad (sufren de alucinaciones).

## Arquitectura base (El Transformador)

En 2017, Google publicó el artículo “Atención es todo lo que necesitas” introduciendo una nueva arquitectura a la cual nombraron Transformer. La idea principal es detener el uso de RNNs y CNNs y, en lugar, utilizar esta arquitectura con el uso de mecanismos de atención pero de una manera muy ingeniosa. Esto permite obtener modelos mucho más paralelizables y, por lo tanto, computacionalmente eficientes comparado con arquitecturas como LSTMs o GRUs mostrando que los Transformadores modelan de mejor forma las dependencias de largo plazo entre las palabras. Se muestra su arquitectura a alto nivel.

![Arquitectura Transformer](https://d2l.ai/_images/transformer.svg)

Stack tecnológico (core)

* Open AI: https://platform.openai.com/
* LangChain: https://python.langchain.com/en/latest/index.html
* Facebook AI Similarity Search: https://ai.facebook.com/tools/faiss/
* Google Text2Speech: https://github.com/pndurette/gTTS
* PyPDF2: https://konfuzio.com/en/pypdf2/
* Gradio: https://gradio.app/

In [1]:
!pip install openai==0.27.7 langchain==0.0.154 faiss-cpu==1.7.4 PyPDF2==3.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install tiktoken==0.4.0 gradio==3.28.3 requests==2.27.1 gTTS==2.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import sys
import requests
import gradio as gr

from gtts import gTTS
from PyPDF2 import PdfReader

from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.callbacks import get_openai_callback
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
path = '/content/gdrive/MyDrive/Colab Notebooks/Inteligencia Artificial/Q&A/'
sys.path.append(path)

In [6]:
from config import api_key_openai

os.environ["OPENAI_API_KEY"] = api_key_openai

In [7]:
def get_pdf_from_url(url):
    response = requests.get(url=url, timeout=120)
    with open('document.pdf', 'wb') as file:
        file.write(response.content)
    return 'document.pdf'

In [8]:
def questiondocument(user_question, url):
    document = get_pdf_from_url(url)
    pdf_reader = PdfReader(document)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
        
    text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len)
    chunks = text_splitter.split_text(text)
      
    embeddings = OpenAIEmbeddings()
    knowledge_base = FAISS.from_texts(chunks, embeddings)
    docs = knowledge_base.similarity_search(user_question)
        
    llm = OpenAI(model_name="gpt-3.5-turbo")
    chain = load_qa_chain(llm, chain_type="stuff")
    with get_openai_callback() as cb:
        response = chain.run(input_documents=docs, question=user_question)

    audio = gTTS(response, lang='es', tld='com.mx', slow=False)
    audio.save("output.mp3")
    cost = cb

    return response, "output.mp3"

In [9]:
demo = gr.Interface(fn=questiondocument, inputs=["text", "text"], outputs=["text", "audio"])
demo.launch(share=True, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e180a3f55fdc170dab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


* Enlace al paper original en formato PDF: https://arxiv.org/pdf/1706.03762.pdf
* Enlace al modo lectura: https://ar5iv.labs.arxiv.org/html/1706.03762

Preguntas sugeridas:

1. ¿Cuál es la estructura básica del modelo Transformer y cómo se compone?
2. ¿Cómo se representa una secuencia de entrada en el modelo Transformer?
3. ¿Cuál es el propósito de la función de atención y cómo se calcula en el Transformer?
4. ¿Cómo se implementa la autoatención en el Transformer y cuáles son sus ventajas?
5. ¿Cuál es la función de las capas de codificación y decodificación en el modelo Transformer?
6. ¿Cómo se lleva a cabo la propagación hacia adelante (forward propagation) en el Transformer?
7. ¿Cuál es el papel de la máscara de atención en el proceso de decodificación?
8. ¿Cómo se realiza el aprendizaje del modelo Transformer y cuál es la función de pérdida utilizada?
9. ¿Cuáles son los hiperparámetros clave que afectan el rendimiento y la capacidad del modelo Transformer?
10. ¿Qué resultados y experimentos se presentan en el artículo para validar la eficacia del modelo Transformer en comparación con otros enfoques existentes?

![Transformador detallado](https://deepfrench.gitlab.io/deep-learning-project/resources/transformer.png)

**Referencias**

* https://deepfrench.gitlab.io/deep-learning-project/
* https://platform.openai.com/docs/models/overview
* https://arxiv.org/pdf/1706.03762.pdf
* https://jalammar.github.io/illustrated-transformer/
* https://youtu.be/n9sLZPLOxG8
* https://gtts.readthedocs.io/en/latest/module.html#module-gtts.lang